In [7]:
import csv
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import sklearn
from sklearn import datasets

In [13]:
# Definisanje klase za opis skupa podataka.
class HabermanDataset(Dataset):
    def __init__(self, path=None):
        super().__init__()
        # Skladiste skupa podataka.
        self.data = []
        
        # Mapiranje imena klasa na numericke vrednosti.
        self.label_index = {
            "Survived-more": 0,
            "Survived-less": 1
        }
        
        # Ucitavanje podataka sa zadate putanje.
        self._load(path)
        
    def _load(self, path):
        with open(path) as fp:
            csv_data = csv.reader(fp)
            for row in csv_data:
                if len(row) > 0:
                    # Parsiranje i skladistenje reda csv datoteke.
                       self.data.append((np.array(np.array(list(map(lambda x: float(x), row[:-1]))), dtype=np.float32), self.label_index[row[-1]]))
        
    def __getitem__(self, index):
        # Ova magicna metoda mora biti implementirana.
        # Predstavlja implementaciju preklapanja operatora
        # indeksiranja. Kao argument prima indeks, a kao povratnu
        # vrednost vraca podatak na zadatom indeksu.
        return self.data[index]
    
    def __len__(self):
        # Ova magicna metoda mora biti implementirana.
        # Povratna vrednost ove metode predstavlja ukupan broj
        # pojava u skupu podataka.
        return len(self.data)

In [14]:
ds = HabermanDataset("haberman.data")
test_loader = DataLoader(ds, shuffle=True, batch_size=30)
train_loader = DataLoader(ds, shuffle=True, batch_size=30)

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, 50)
        self.fc2 = nn.Linear(50, 20)
        self.fc3 = nn.Linear(20, 10)
        self.fc4 = nn.Linear(10, 3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return x

In [16]:
net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 30 == 10:
            print('%d. epoch loss: %.3f' % (epoch + 1, running_loss / 5))
            running_loss = 0.0

print("Obucavanje zavrseno.")

1. epoch loss: 1.967
2. epoch loss: 1.700
3. epoch loss: 1.748
4. epoch loss: 1.549
5. epoch loss: 1.235
Obucavanje zavrseno.


In [17]:
total = 0
correct = 0

with torch.no_grad():
    for data in test_loader:
        features, labels = data
        outputs = net(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("Tacnost: {:.3f}%".format(100 * correct / total))

Tacnost: 75.490%


In [61]:
ds.__len__()

306

In [47]:
#prvi podatak
ds.__getitem__(0)

(array([30., 64.,  3.], dtype=float32), 0)

In [48]:
#poslednji
ds.__getitem__(305)

(array([83., 58.,  4.], dtype=float32), 1)